In [2]:
import pymongo 
from pymongo import MongoClient

In [3]:
# connect to database
connection = MongoClient('localhost', 27017)

In [23]:
import json

from pprintpp import pprint as pp

In [46]:
def insert_tweet(tweet_json):
    tweets.replace_one({"_id" : tweet_json["id"]}, tweet_json, upsert=True)
    
    user_json = tweet_json['user']
    users.replace_one({"_id" : user_json["id"]}, user_json, upsert=True)    
    
    for user_mencioned in tweet_json['user_mentions']:
        user_mencioned["_id"] = user_mencioned["id"]
        del(user_mencioned["id"])
        try:
            users.insert_one(user_mencioned)
        except:
            continue
        
    if 'retweeted_status' in tweet_json:
        insert_tweet(tweet_json['retweeted_status'])

In [47]:
import json

connection.drop_database('twitter')
db = connection.twitter

tweets = db.tweets
users = db.users

tweets_data_path = '../data/tweets.json'

tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    tweet_json = json.loads(line)
    insert_tweet(tweet_json)
